# **Laboratorio 5 Redes Neuronales con Pytorch**
Arancibia Aguilar Daniel Andree

Ingeniería en Ciencias de la Computación

Para las redes neuronales hare uso del siguiente dataset:

https://www.kaggle.com/datasets/anseldsouza/water-pump-rul-predictive-maintenance

Convertido a una clasificación de 9 clases

Haciendo las pruebas con diferentes hiperparametros para encontrar el mas optimo

In [46]:
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm  # For nice progress bar!

%matplotlib inline

In [47]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [48]:
# Here we create our simple neural network. For more details here we are subclassing and
# inheriting from nn.Module, this is the most general way to create your networks and
# allows for more flexibility. I encourage you to also check out nn.Sequential which
# would be easier to use in this scenario but I wanted to show you something that
# "always" works.
class RedNeuronalMLS(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RedNeuronalMLS, self).__init__()
        # Our first linear layer take input_size, in this case 784 nodes to 50
        # and our second linear layer takes 50 to the num_classes we have, in
        # this case 10.
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        """
        x here is the mnist images and we run it through fc1, fc2 that we created above.
        we also add a ReLU activation function in between and for that (since it has no parameters)
        I recommend using nn.functional (F)
        """
        x = self.fc1(x)
        x = F.sigmoid(x)
        # x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

**Preproceso el dataset convirtiendolo a una clasificación one vs all**

In [92]:
url = "/content/gdrive/MyDrive/SIS420/Lab5/rul_hrs.csv"
dataframe = pd.read_csv(url)
# Definir los rangos y los valores a los que se deben convertir
rango_valores = [
    (range(0, 84), 0),
    (range(84, 167), 1),
    (range(167, 251), 2),
    (range(251, 334), 3),
    (range(334, 418), 4),
    (range(418, 502), 5),
    (range(502, 585), 6),
    (range(585, 669), 7),
    (range(669, 753), 8),
    (range(753, 838), 9)
]
# Iterar sobre los rangos y valores y reemplazar los valores en la columna "rul"
for rango, valor in rango_valores:
    dataframe['rul'] = dataframe['rul'].replace(rango, valor)

print(dataframe)

        Unnamed: 0  timestamp  sensor_00  sensor_01  sensor_02  sensor_03  \
0                0      43191          2         47         53         46   
1                1      43191          2         47         53         46   
2                2      43191          2         47         53         46   
3                3      43191          2         47         53         46   
4                4      43191          2         47         53         46   
...            ...        ...        ...        ...        ...        ...   
166436      166436      43307          2         46         53         44   
166437      166437      43307          2         46         53         44   
166438      166438      43307          2         46         53         44   
166439      166439      43307          2         46         53         44   
166440      166440      43307          2         46         53         44   

        sensor_04  sensor_05  sensor_06  sensor_07  ...  sensor_42  sensor_

**Separo las caracteristicas**

In [93]:
X = dataframe.drop(columns=['rul'])
y = dataframe['rul']
print(X.shape)
print(y.shape)
X=X.values
y=y.values

(166441, 52)
(166441,)


Normalizamos para tener una mejor precisión en el entrenamiento

In [94]:
def featureNormalize(X):
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    epsilon = 1e-8
    sigma += epsilon
    X_norm = (X - mu) / sigma
    return X_norm, mu, sigma

In [95]:
X, mu, sigma = featureNormalize(X)
print(X)

[[-1.7320404  -1.73196589 -0.23395421 ...  0.56148079  1.07881881
  -0.00905751]
 [-1.73201959 -1.73196589 -0.23395421 ...  0.56148079  1.07881881
  -0.00905751]
 [-1.73199878 -1.73196589 -0.23395421 ...  0.53020102  1.00861677
   0.01600662]
 ...
 [ 1.73199878  1.74447716 -0.23395421 ...  1.98471052  1.35962696
   0.04107075]
 [ 1.73201959  1.74447716 -0.23395421 ...  1.92215097  1.429829
   0.06613488]
 [ 1.7320404   1.74447716 -0.23395421 ...  1.68755266  1.21922289
   0.03271604]]


**Separo los valores de entrenamiento y los valores de prueba**

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y = np.squeeze(y)
print(X_train.shape)
print(y_train.shape)
print(X.shape)
print(y.shape)
y = np.squeeze(y)
print(y)

(133152, 52)
(133152,)
(166441, 52)
(166441,)
[3 3 3 ... 0 0 0]


In [54]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Crear conjuntos de datos y DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 64

# DataLoader para datos de entrenamiento
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# DataLoader para datos de prueba
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# **Prueba 1 de hiperparametros:**

In [55]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 52
num_classes = 10
learning_rate = 0.001
batch_size = 500000
num_epochs = 3

cpu


In [56]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)

In [57]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=52, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [58]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [59]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # for batch_idx, (data, targets) in enumerate(train_loader):

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape)
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # print("-"*30)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

  0%|          | 1/2081 [00:00<03:42,  9.36it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])


  2%|▏         | 37/2081 [00:00<00:09, 208.63it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])


  3%|▎         | 71/2081 [00:00<00:07, 267.07it/s]

...

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])


 98%|█████████▊| 2044/2081 [00:05<00:00, 341.14it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])


100%|██████████| 2081/2081 [00:06<00:00, 342.80it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([32, 52])


In [60]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Precisión en el entrenamiento: {p_train*100:.2f}")
print(f"Precisión en el test: {p_test*100:.2f}")

Precisión en el entrenamiento: 86.85
Precisión en el test: 86.86


# **Prueba 2 con otros hiperparametros**

In [61]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 52
num_classes = 10
learning_rate = 0.004
batch_size = 400000
num_epochs = 3

cpu


In [62]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)

In [63]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=52, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [64]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [65]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # for batch_idx, (data, targets) in enumerate(train_loader):

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape)
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # print("-"*30)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

  1%|▏         | 28/2081 [00:00<00:07, 272.63it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])


  3%|▎         | 61/2081 [00:00<00:06, 303.70it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])


  5%|▍         | 96/2081 [00:00<00:06, 321.86it/s]

...

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])


 99%|█████████▉| 2057/2081 [00:07<00:00, 226.80it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])


100%|██████████| 2081/2081 [00:08<00:00, 259.39it/s]

torch.Size([32, 52])


In [66]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Precisión en el entrenamiento: {p_train*100:.2f}")
print(f"Precisión en el test: {p_test*100:.2f}")

Precisión en el entrenamiento: 96.02
Precisión en el test: 95.96


# **Prueba 3 con otros hiperparametros**

In [85]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 52
num_classes = 10
learning_rate = 0.8
batch_size = 800000
num_epochs = 3

cpu


In [86]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)

In [87]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=52, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [88]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [89]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # for batch_idx, (data, targets) in enumerate(train_loader):

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape)
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # print("-"*30)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

  3%|▎         | 71/2081 [00:00<00:05, 350.53it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

  5%|▌         | 107/2081 [00:00<00:05, 331.49it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

  9%|▊         | 177/2081 [00:00<00:05, 324.83it/s]

...

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])


 99%|█████████▊| 2052/2081 [00:07<00:00, 309.48it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

100%|██████████| 2081/2081 [00:07<00:00, 270.41it/s]

torch.Size([32, 52])


In [90]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Precisión en el entrenamiento: {p_train*100:.2f}")
print(f"Precisión en el test: {p_test*100:.2f}")

Precisión en el entrenamiento: 63.33
Precisión en el test: 62.74


# **Resultado de las 3 pruebas**
## **Prueba 1 con los hiperparametros**

learning_rate = 0.001 batch_size = 500000

**Precisión en el entrenamiento: 86.85**

**Precisión en el test: 86.86**
## **Prueba 2 con los hiperparametros**

learning_rate = 0.004 batch_size = 400000

**Precisión en el entrenamiento: 96.02**

**Precisión en el test: 95.96**

## **Prueba 3 con los hiperparametros**

learning_rate = 0.8 batch_size = 800000

**Precisión en el entrenamiento: 63.33**

**Precisión en el test: 62.74**

Con el learning rate y un batch size bajo se obtuvo la mejor precisión en las 3 pruebas, teniendo una mejora de 10% en comparación a la prueba 1, en la prueba 3 subiendo el learning rate y el batch size tuvimos una gran disminución en la precisión